RAG Pipeline-Data Ingestion to Vector DB

In [4]:
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
loader=DirectoryLoader("data/pdf_files", glob="*.pdf",loader_cls=PyMuPDFLoader)
loader.load()

[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-10-23T08:55:46+00:00', 'source': 'data\\pdf_files\\sravan_potnuru.pdf', 'file_path': 'data\\pdf_files\\sravan_potnuru.pdf', 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'Copy of Sravan potnuru', 'author': 'Sravan Potnuru', 'subject': '', 'keywords': 'DAG2irj_3ks,BAErSMjiGi8,0', 'moddate': '2025-10-23T08:55:45+00:00', 'trapped': '', 'modDate': "D:20251023085545+00'00'", 'creationDate': "D:20251023085546+00'00'", 'page': 0}, page_content='P R O F I L E  S U M M A R Y\nA highly motivated and adaptable AI/ML enthusiast with hands-on experience in full-stack development and cloud-based solutions.\nSkilled in React JS, Spring Boot, and MySQL, capable of designing and maintaining scalable, three-tier architectures with RESTful\nAPIs. Gained practical expertise through an AI/ML internship, working on real-time video surveillance and facial recognition systems,\nleveraging FastAPI, Python, Triton Inference Ser

In [3]:
import os
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader
from pathlib import Path
# Try this first (newer LangChain versions)
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    # Fallback for older versions
    from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
def read_pdf_files(pdf_dir):
    all_documents=[]
    pdf_dir=Path(pdf_dir)
    pdf_files=list(pdf_dir.glob("*.pdf"))
    print(len(pdf_files), " pdf file/s founded")
    for file in pdf_files:
        print("file name:",file.name)
        loader=PyMuPDFLoader(str(file))
        documents=loader.load()
        for doc in documents:
            doc.metadata['source_file']=file.name
            doc.metadata['file_type']="pdf"
        all_documents.extend(documents)
        print(len(all_documents))
    return all_documents
all_pdf_documents=read_pdf_files("data/pdf_files")
all_pdf_documents




1  pdf file/s founded
file name: sravan_potnuru.pdf
2


[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-10-23T08:55:46+00:00', 'source': 'data\\pdf_files\\sravan_potnuru.pdf', 'file_path': 'data\\pdf_files\\sravan_potnuru.pdf', 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'Copy of Sravan potnuru', 'author': 'Sravan Potnuru', 'subject': '', 'keywords': 'DAG2irj_3ks,BAErSMjiGi8,0', 'moddate': '2025-10-23T08:55:45+00:00', 'trapped': '', 'modDate': "D:20251023085545+00'00'", 'creationDate': "D:20251023085546+00'00'", 'page': 0, 'source_file': 'sravan_potnuru.pdf', 'file_type': 'pdf'}, page_content='P R O F I L E  S U M M A R Y\nA highly motivated and adaptable AI/ML enthusiast with hands-on experience in full-stack development and cloud-based solutions.\nSkilled in React JS, Spring Boot, and MySQL, capable of designing and maintaining scalable, three-tier architectures with RESTful\nAPIs. Gained practical expertise through an AI/ML internship, working on real-time video surveillance and facial recognition s

In [ ]:
def split_document(documents,chunk_size=1000,chunk_overlap=100):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                                 chunk_overlap=chunk_overlap, 
                                                 length_function=len,
                                                 separators=["\n\n", "\n", " ", ""])
    split_docs=text_splitter.split_documents(documents)
    print(f"splitted {len(all_pdf_documents)} into {len(split_docs)} chunks")
    print(split_docs)
    return split_docs
chunks=split_document(all_pdf_documents)


splitted 2 into 7 chunks
[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-10-23T08:55:46+00:00', 'source': 'data\\pdf_files\\sravan_potnuru.pdf', 'file_path': 'data\\pdf_files\\sravan_potnuru.pdf', 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'Copy of Sravan potnuru', 'author': 'Sravan Potnuru', 'subject': '', 'keywords': 'DAG2irj_3ks,BAErSMjiGi8,0', 'moddate': '2025-10-23T08:55:45+00:00', 'trapped': '', 'modDate': "D:20251023085545+00'00'", 'creationDate': "D:20251023085546+00'00'", 'page': 0, 'source_file': 'sravan_potnuru.pdf', 'file_type': 'pdf'}, page_content='P R O F I L E  S U M M A R Y\nA highly motivated and adaptable AI/ML enthusiast with hands-on experience in full-stack development and cloud-based solutions.\nSkilled in React JS, Spring Boot, and MySQL, capable of designing and maintaining scalable, three-tier architectures with RESTful\nAPIs. Gained practical expertise through an AI/ML internship, working on real-time video surveillance

Embeddings generator

In [25]:
import numpy as np 
from sentence_transformers import SentenceTransformer 
import chromadb 
from chromadb.config import Settings 
import uuid 
from typing import List, Dict, Any, Tuple 
from sklearn.metrics.pairwise import cosine_similarity

In [68]:
class EmbeddingManager:
     """Handles document embedding generation using SentenceTransformer""" 
     def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
         #Initialize the embedding manager Args: model name: HuggingFace model name for sentence embeddings 
         self.model_name= model_name 
         self.model=None 
         self._load_model()
     def _load_model(self): 
        """Load the SentenceTransformer model""" 
        try: 
            print(f"Loading embedding model: {self.model_name}") 
            self.model=SentenceTransformer(self.model_name) 
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}") 
        except Exception as e: 
            print(f"Error loading model {self.model_name}: {e}") 
     def generate_embeddings(self,texts:List[str]) :
         if not self.model:
             print("model not loaded")
         embeddings=self.model.encode(texts,show_progress_bar=True)
         print(f"length of embeddings are {len(embeddings)}")
         return embeddings
embedding_manager=EmbeddingManager()

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


Vector store

In [102]:
class VectorStore:
    def __init__(self,collection_name:str="pdf_documents",persistent_directory:str="vector_store"):
        self.collection_name=collection_name
        self.persistent_directory=persistent_directory
        self.collection=None
        self.client=None
        self._initialize_store()
    def _initialize_store(self):
        os.makedirs(self.persistent_directory,exist_ok=True)
        self.client=chromadb.PersistentClient(path=self.persistent_directory)
        self.collection=self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description":"Am creating the collection using ChromaDB"}
            
        )
        print(f"collection is created with name: {self.collection_name}")
        # print(f"number of documents in {self.collection_name}")
    def add_documents(self,documents:List[Any],embeddings:np.ndarray):
        if len(documents)!=len(embeddings):
            return None
        ids=[]
        metadatas=[]
        documents_text=[]
        embeddings_list=[]
        for i,(doc,embedding) in enumerate(zip(documents,embeddings)):
            #Generate unique ID 
            doc_id=f"doc_(uuid.uuid4().hex[:8])_{i}" 
            ids.append(doc_id) 
            # Prepare metadata 
            metadata=dict(doc.metadata) 
            metadata['doc_index'] = i 
            metadata['content_length'] = len(doc.page_content) 
            metadatas.append(metadata) 
            #Document content 
            documents_text.append(doc.page_content) 
            # Embedding
            embeddings_list.append(embedding.tolist())
            try:
                self.collection.add(
                    ids=ids,
                    embeddings=embeddings_list,
                    metadatas=metadatas,
                    documents=documents_text
                )
                print(f"successfully added the document to the {self.collection_name}")
                print(f"Total documents in the colllection: {self.collection.count()}")
            except:
                print("Error adding documents to the store")
vector_store=VectorStore()

collection is created with name: pdf_documents


In [103]:
texts=[doc.page_content for doc in chunks]
print("Text chunks are; ",texts)
len(texts)

Text chunks are;  ['P R O F I L E  S U M M A R Y\nA highly motivated and adaptable AI/ML enthusiast with hands-on experience in full-stack development and cloud-based solutions.\nSkilled in React JS, Spring Boot, and MySQL, capable of designing and maintaining scalable, three-tier architectures with RESTful\nAPIs. Gained practical expertise through an AI/ML internship, working on real-time video surveillance and facial recognition systems,\nleveraging FastAPI, Python, Triton Inference Server, and LLM technologies. Experienced in deploying microservices on AWS,\nautomating pipelines with Lambda, S3, and Kinesis Video Streams, and delivering solutions for government surveillance projects.\nPassionate about combining AI, cloud, and full-stack development to create impactful, real-world applications.\nW O R K  E X P E R I E N C E\nAI/ML Developer Intern\nParabola9\n12/2024 – Present | Nuzvid\nWorking in a microservices-based AI/ML backend team focused on cloud deployments, model inference'

7

In [97]:
embeddings=embedding_manager.generate_embeddings(texts)
print(embeddings)

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.48it/s]

length of embeddings are 7
[[-0.09921841 -0.09519143  0.02601445 ... -0.05196509 -0.05783535
  -0.04707638]
 [-0.08265381 -0.0581031   0.01882411 ... -0.02749567  0.01231377
  -0.06328682]
 [-0.08673339 -0.03277342 -0.01005095 ...  0.00216525 -0.05772281
   0.02977197]
 ...
 [-0.13217147  0.01362875 -0.00745958 ... -0.08978795 -0.06070377
  -0.0275821 ]
 [-0.12108929  0.00088403 -0.07094926 ... -0.09723766 -0.00922065
   0.05233534]
 [-0.06347242  0.07656928  0.01047438 ...  0.12074647 -0.0656387
   0.06181148]]


In [98]:
vector_store.add_documents(chunks,embeddings)

successfully added the document to the pdf_documents
Total documents in the colllection: 7
successfully added the document to the pdf_documents
Total documents in the colllection: 7
successfully added the document to the pdf_documents
Total documents in the colllection: 7
successfully added the document to the pdf_documents
Total documents in the colllection: 7
successfully added the document to the pdf_documents
Total documents in the colllection: 7
successfully added the document to the pdf_documents
Total documents in the colllection: 7
successfully added the document to the pdf_documents
Total documents in the colllection: 7


Creating RAG Retrieval

In [113]:
class RAGRetriever:
    def __init__(self,vectorstore:VectorStore,embedding_manager:EmbeddingManager):
        self.vectorstore=vectorstore
        self.embedding_manager=embedding_manager
    def retrieve(self,query:str,top_k:int=5,similarity_threshold:float=0.0):
        print(f"Retrieving documents for the query: {query}")
        print(f"default top_k is:{top_k} and score_threshold is {similarity_threshold}")
        query_embedding=self.embedding_manager.generate_embeddings([query])[0]
        # print(query_embedding)
        # print([query_embedding.tolist()])
        try:
            results=self.vectorstore.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            print(results)
            retrieved_docs=[]
            if results['documents'] and results['documents'][0]:
                print(f"Retrieving results for the query:{query}")
                documents=results['documents'][0]
                metadatas=results['metadatas'][0]
                distances=results['distances'][0]
                ids=results['ids'][0]
                for i,(document,metadata,id,distance) in enumerate(zip(documents,metadatas,ids,distances)):
                    cosine=1-distance
                    if cosine<=similarity_threshold:
                        retrieved_docs.append(
                            {
                                "id":id,
                                "content":document,
                                "metadata":metadata,
                                "similarity_score":cosine,
                                "distance":distance,
                                "rank":i+1
                            }
                        )
                print(f"Retrieved {len(retrieved_docs)} after filtering")
            else:
                print("No records found")
            return retrieved_docs
        except:
            print("failed retrieving the results")

rag_retriever=RAGRetriever(vector_store,embedding_manager)
retrieved_results=rag_retriever.retrieve("what is the education")
# print(retrieved_results)
for i,result in enumerate(retrieved_results):
    print("------------------------------------------------------------------")
    print(f"Result: {i+1}: {result['content']}")

Retrieving documents for the query: what is the education
default top_k is:5 and score_threshold is 0.0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 45.40it/s]

length of embeddings are 1
{'ids': [['doc_(uuid.uuid4().hex[:8])_4', 'doc_(uuid.uuid4().hex[:8])_5', 'doc_(uuid.uuid4().hex[:8])_6', 'doc_(uuid.uuid4().hex[:8])_0', 'doc_(uuid.uuid4().hex[:8])_3']], 'embeddings': None, 'documents': [["C E R T I F I C A T I O N\nSuccessful completion certificate on Java Development Internship.\nCertification on 7-Days Bootcamp on Basic Web-Dev With Bootstrap.\nCertification on completion of JavaScript and React JS bootcamp.\nCertification on completion on Resume, Networking and Interview Skills.\nC O - C U R R I C U L A R\nAchieved NCC 'B' Certificate and 'C' Certificate.\nCertification on completion on Resume, Networking and Interview Skills.\nCertificate of Merit for All India Essay Writing Event 2018.\nCertificate of Participation and Certificate of Appreciation for National Level Techno Management Fest \nCertificate of Participation for All India Inter-University Yoga Championship (2019-2020)\nC O N T A C T\n+91 9391872342\nsravanpotnuru24@gmail.com